# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
import pickle
from sklearn.svm import SVC

In [2]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\EL7md\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\EL7md\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine=create_engine('sqlite:///InsertDatabaseName.db')
df=pd.read_sql_table("messages_disaster", con=engine)



In [4]:
df.head()

,id,message,original,genre,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X=df['message']
Y=df.drop(['id','message','original','genre'],axis=1)


In [6]:
X.shape

(26216,)

In [7]:
Y.shape

(26216, 36)

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    words=word_tokenize(text)
    lem=WordNetLemmatizer()
    tokens=[]
    for i in words:
        tokens.append(lem.lemmatize(i).lower().strip())
    return tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline=Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                 ('tfidf',TfidfTransformer()),
                 ('clf',MultiOutputClassifier(RandomForestClassifier()))
    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.3, random_state = 0)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001E38F781048>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
y_preds= pipeline.predict(X_test)
for i,col in enumerate(y_test):   
    print(col)
    print(classification_report(y_test[col],y_preds[:, i]))
    

('related',)
              precision    recall  f1-score   support

           0       0.71      0.26      0.38      1787
           1       0.81      0.97      0.88      6028
           2       0.82      0.18      0.30        50

    accuracy                           0.80      7865
   macro avg       0.78      0.47      0.52      7865
weighted avg       0.79      0.80      0.77      7865

('request',)
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      6481
           1       0.89      0.41      0.56      1384

    accuracy                           0.89      7865
   macro avg       0.89      0.70      0.75      7865
weighted avg       0.89      0.89      0.87      7865

('offer',)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       0.00      0.00      0.00        27

    accuracy                           1.00      7865
   macro avg       0.50      0.50    

C:\Users\EL7md\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



('water',)
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7361
           1       0.89      0.23      0.37       504

    accuracy                           0.95      7865
   macro avg       0.92      0.62      0.67      7865
weighted avg       0.95      0.95      0.93      7865

('food',)
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6999
           1       0.90      0.35      0.50       866

    accuracy                           0.92      7865
   macro avg       0.91      0.67      0.73      7865
weighted avg       0.92      0.92      0.91      7865

('shelter',)
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      7157
           1       0.91      0.24      0.38       708

    accuracy                           0.93      7865
   macro avg       0.92      0.62      0.67      7865
weighted avg       0.93      0.93      

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = {'tfidf__use_idf': (True, False),
              'clf__estimator__min_samples_leaf': [1,5],
              'clf__estimator__min_samples_split': [2,5],
              'clf__estimator__n_estimators' :[50,100]}

In [15]:
cv=GridSearchCV(pipeline, param_grid=parameters)

In [16]:
cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
cv.fit(X_train,y_train)
print(cv.best_params_)

{'clf__estimator__min_samples_leaf': 1, 'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100, 'tfidf__use_idf': False}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
#Improve  the pipeline
pipeline2=Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    
])
    



In [27]:
#Train & predict
pipeline2.fit(X_train,y_train)
ypreds= pipeline2.predict(X_test)
for j,col2 in enumerate(y_test):   
    print(col2)
    print(classification_report(y_test[col2],y_preds[:, j]))


('related',)
              precision    recall  f1-score   support

           0       0.71      0.26      0.38      1787
           1       0.81      0.97      0.88      6028
           2       0.82      0.18      0.30        50

    accuracy                           0.80      7865
   macro avg       0.78      0.47      0.52      7865
weighted avg       0.79      0.80      0.77      7865

('request',)
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      6481
           1       0.89      0.41      0.56      1384

    accuracy                           0.89      7865
   macro avg       0.89      0.70      0.75      7865
weighted avg       0.89      0.89      0.87      7865

('offer',)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       0.00      0.00      0.00        27

    accuracy                           1.00      7865
   macro avg       0.50      0.50    

C:\Users\EL7md\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



('child_alone',)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7865

    accuracy                           1.00      7865
   macro avg       1.00      1.00      1.00      7865
weighted avg       1.00      1.00      1.00      7865

('water',)
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7361
           1       0.89      0.23      0.37       504

    accuracy                           0.95      7865
   macro avg       0.92      0.62      0.67      7865
weighted avg       0.95      0.95      0.93      7865

('food',)
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6999
           1       0.90      0.35      0.50       866

    accuracy                           0.92      7865
   macro avg       0.91      0.67      0.73      7865
weighted avg       0.92      0.92      0.91      7865

('shelter',)
              precisi


('other_weather',)
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7432
           1       0.57      0.02      0.04       433

    accuracy                           0.95      7865
   macro avg       0.76      0.51      0.50      7865
weighted avg       0.93      0.95      0.92      7865

('direct_report',)
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      6293
           1       0.86      0.35      0.49      1572

    accuracy                           0.86      7865
   macro avg       0.86      0.67      0.71      7865
weighted avg       0.86      0.86      0.83      7865



### 9. Export your model as a pickle file

In [28]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.